In [15]:
import os
from PIL import Image

# Define the classes
classes = ['cherry', 'strawberry', 'tomato']
data_dir = './train_data'

# Dictionary to store the loaded images
data = {}

for class_name in classes:
    class_dir = os.path.join(data_dir, class_name)
    images = []
    
    # Loop through all files in the class directory
    for file_name in os.listdir(class_dir):
        if file_name.endswith(('.jpg')):  # Check for image files
            file_path = os.path.join(class_dir, file_name)
            
            # Open the image and append it to the list
            img = Image.open(file_path)
            images.append(img)
    
    # Store images for this class
    data[class_name] = images

# Example: Accessing images from the 'cherry' class
print(f'Loaded {len(data["cherry"])} images from cherry class.')
print(f'Loaded {len(data["strawberry"])} images from strawberry class.')
print(f'Loaded {len(data["tomato"])} images from tomato class.')

Loaded 1495 images from cherry class.
Loaded 1495 images from strawberry class.
Loaded 1495 images from tomato class.
